In [ ]:
import torch
import matplotlib.pyplot as plt

# Load the loss data
data1 = torch.load("/root/ovd-attack/results/loss/dino/dino_single_losses.pt")
data2 = torch.load("/root/ovd-attack/results_iou_clamp_8p/loss/dino/dino_single_losses.pt")

# Set the target length
target_length = 50000

# Pad data1
if len(data1) < target_length:
    padding1 = torch.zeros(target_length - len(data1))
    data1 = torch.cat([data1, padding1])

# Pad data2
if len(data2) < target_length:
    padding2 = torch.zeros(target_length - len(data2))
    data2 = torch.cat([data2, padding2])

ymin = min(data1.min().item(), data2.min().item())
ymax = max(data1.max().item(), data2.max().item())

# Create subplots: 2 rows, 1 column
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# First subplot
axes[0].plot(data1, color='blue')
axes[0].set_title("Single Prompt Loss")
axes[0].set_ylabel("Loss")
axes[0].set_ylim(ymin, ymax)
axes[0].grid(True)

# Second subplot
axes[1].plot(data2, color='green')
axes[1].set_title("Multi Prompt Loss")
axes[1].set_xlabel("Iteration")
axes[1].set_ylabel("Loss")
axes[1].set_ylim(ymin, ymax)
axes[1].grid(True)

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
texts_list = ['car', 'automobile', 'ride', 'wheels', 'transport']
results_dict = {
    text: {
        'clean': {},
        'adv': {}
    }
    for text in texts_list
}
print(results_dict)
results_dict['car']['clean']['da1111.npz'] = True
results_dict['car']['adv']['da1112.npz'] = False
print(results_dict)

In [ ]:
import torch

def cal_asr_all(pt_file_path):
    """
    {
        "car": {"clean": {...}, "adv": {...}},
        "automobile": {"clean": {...}, "adv": {...}},
        ...
    }
    """
    results_dict = torch.load(pt_file_path)

    clean_result = {}
    adv_result = {}

    for text, result in results_dict.items():
        clean_dict = result.get("clean", {})
        adv_dict = result.get("adv", {})

        for filename, v in clean_dict.items():
            clean_result[filename] = clean_result.get(filename, False) or v

        for filename, v in adv_dict.items():
            adv_result[filename] = adv_result.get(filename, False) or v

    clean_true_count = sum(1 for v in clean_result.values() if v)
    adv_true_count = sum(1 for v in adv_result.values() if v)

    asr = (clean_true_count - adv_true_count) / max(clean_true_count, 1)

    print(f"Clean detections: {clean_true_count}")
    print(f"Adv detections: {adv_true_count}")
    print(f"ASR: {asr:.2%}")

    return clean_true_count, adv_true_count, asr


for score in [0.1]:
    _, _, res_single = cal_asr_all(f'/root/ovd-attack/results/adv/dino/train_multi/dino_multi_{score}.pt')
print('*' * 40)
for score in [0.1]:
    _, _, res_single = cal_asr_all(f'/root/ovd-attack/results/adv/dino/train_single/dino_multi_{score}.pt')


# 置信度设置
# dino: [0.3, 0.5]
# glip: [0.01, 0.1, 0.3, 0.5]
# yolo: [0.01, 0.1, 0.3, 0.5]

# for score in [0.1, 0.2, 0.3, 0.4, 0.5]:
#     _, _, res_single = cal_asr_all(f'/root/ovd-attack/results/adv/dino/train_multi/dino_multi_{score}.pt')
# print('*' * 40)
# for score in [0.1, 0.2, 0.3, 0.4, 0.5]:
#     _, _, res_single = cal_asr_all(f'/root/ovd-attack/results/adv/dino/train_single/dino_multi_{score}.pt')

# print('#' * 40)
# for score in [0.1, 0.2, 0.3, 0.4, 0.5]:
#     _, _, res_single = cal_asr_all(f'/root/ovd-attack/results/adv/glip/train_multi/glip_multi_{score}.pt')
# print('*' * 40)
# for score in [0.1, 0.2, 0.3, 0.4, 0.5]:
#     _, _, res_single = cal_asr_all(f'/root/ovd-attack/results/adv/glip/train_single/glip_multi_{score}.pt')

# print('#' * 40) 
# for score in [0.1, 0.2, 0.3, 0.4, 0.5]:
#     _, _, res_single = cal_asr_all(f'/root/ovd-attack/results/adv/yolo/train_multi/yolo_multi_{score}.pt')
# print('*' * 40)
# for score in [0.1, 0.2, 0.3, 0.4, 0.5]:
#     _, _, res_single = cal_asr_all(f'/root/ovd-attack/results/adv/yolo/train_single/yolo_multi_{score}.pt')



In [ ]:
import torch

def cal_asr_for_category(pt_file_path, category_name):
    """
    Args:
        pt_file_path (str)
        category_name (str)
    return:
        clean_true_count (int)
        adv_true_count (int)
        asr (float)
    """
    results_dict = torch.load(pt_file_path)

    if category_name not in results_dict:
        raise ValueError(
            f"Class {category_name} can't be found, available classes: {list(results_dict.keys())}")

    clean_dict = results_dict[category_name].get("clean", {})
    adv_dict = results_dict[category_name].get("adv", {})

    clean_true_count = sum(1 for v in clean_dict.values() if v)
    adv_true_count = sum(1 for v in adv_dict.values() if v)

    asr = (clean_true_count - adv_true_count) / max(clean_true_count, 1)

    print(f"[{category_name}] Clean detections: {clean_true_count}")
    print(f"[{category_name}] Adv detections: {adv_true_count}")
    print(f"[{category_name}] ASR: {asr:.2%}")

    return clean_true_count, adv_true_count, asr

_, _, res_single = cal_asr_for_category('/root/ovd-attack/results/adv/dino/train_single/dino_multi_0.1.pt', 'combat vehicle')

In [ ]:
import torch

data = torch.load('/root/ovd-attack/results/adv/dino/train_single/dino_multi_0.1.pt')
base = "vehicle"
others = ["car", "drive", "wheels"]

diff_files = {}

image_ids = data[base]['adv'].keys()

for img_id in image_ids:
    base_adv = data[base]['adv'][img_id]

    for text in others:
        other_adv = data[text]['adv'].get(img_id)

        if other_adv is None or other_adv != base_adv:
            diff_files.setdefault(img_id, []).append(text)

for img_id, diff_texts in diff_files.items():
    print(f"Image {img_id} differs in: {diff_texts}")


In [ ]:
import torch

data = torch.load('/root/ovd-attack/results/adv/dino/train_single/dino_multi_0.1.pt')
print(data['vehicle']['clean']['pos_0_az270_0_el-90_0_dist30_0.npz'])

In [ ]:
import gensim.downloader as api
from gensim.models import KeyedVectors

model_path = '/root/ovd-attack/weights/word2vec/GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(model_path, binary=True)

similar_words = model.most_similar('vehicle', topn=10)

for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")


In [ ]:
import torch

data = torch.load('/root/ovd-attack/results/adv/dino/train_single/dino_multi_0.1.pt')
print(data.keys())
for text in data.keys():
    print(f"Text: {text}")
    print(data[text].keys())

In [ ]:
def loss_smooth_img(img, mask):
    img = img.unsqueeze(0) / 255.0
    s1 = torch.pow(img[:, :, 1:, :-1] - img[:, :, :-1, :-1], 2)
    s2 = torch.pow(img[:, :, :-1, 1:] - img[:, :, :-1, :-1], 2)
    mask = mask[:, :-1, :-1]
    mask = mask.unsqueeze(1)
    return 1e-4 * torch.sum(mask * (s1 + s2))